In [1]:
import objgraph 
import pygame
from Projectile import Projectile, Bomb
from Enemy import Enemy, EnemyFactory
from Player import Player
from  Weapon import WeaponFactory
from Weapon import Weapon
import gc

pygame 2.5.2 (SDL 2.28.3, Python 3.11.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class NoFlyWeightEnemy():
    projectiles = pygame.sprite.Group()
    def __init__(self, name: str, image: pygame.Surface, weapon_name: str, speed: float, 
                 default_health: int, pos: tuple[float, float]):
        super().__init__()
        self.image = pygame.transform.scale(pygame.image.load("./Images/smallEnemy.png"), (50,50))
        self.rect = self.image.get_rect()
        self.speed = speed
        self.weapon = WeaponFactory.get(weapon_name)

        self.name = name
        self.health = default_health
        self.movement_vector = [0, 0]
        self.pos = list(pos)
        self.last_shot_time = pygame.time.get_ticks()
        
    def move(self, enemies: pygame.sprite.Group, player_pos: tuple[float, float], tDelta: float):
        self.movement_vector = (player_pos[0] - self.pos[0],
                               player_pos[1] - self.pos[1])
        self.movement_vector = Weapon.normalize_vector(self.movement_vector)
        self.pos[0] += self.movement_vector[0] * self.speed * tDelta
        self.pos[1] += self.movement_vector[1] * self.speed * tDelta
        
        # Collision test with other enemies
        self.movement_vector = [0, 0]
        for sprite in enemies:
            if sprite is self:
                continue
            if pygame.sprite.collide_circle(self, sprite):
                self.movement_vector[0] += self.pos[0] - sprite.pos[0]
                self.movement_vector[1] += self.pos[1] - sprite.pos[1]

        self.movement_vector = Weapon.normalize_vector(self.movement_vector)
        self.pos[0] += self.movement_vector[0] * 0.5  # The constant is how far the sprite will be
        self.pos[1] += self.movement_vector[1] * 0.5  # dragged from the sprite it collided with
        
        self.rect.topleft = self.pos

    def attack(self, target_pos):
        self.weapon.attack(self, target_pos, self.last_shot_time)
        self.last_shot_time = pygame.time.get_ticks()

    def render(self, surface):
        surface.blit(self.image, self.pos)
    

In [3]:
# Display memory growth before creating instances
print("Memory usage before creating instances:")
objgraph.show_growth()

# create more instances of each enemy type
for i in range(1000):
    enemy = Enemy(name = "small", pos=(0,0))

# Display memory growth after creating instances
print("Memory usage after creating instances:")
objgraph.show_growth()


Memory usage before creating instances:
function                      21788    +21788
tuple                         12286    +12286
dict                          10792    +10792
list                           5899     +5899
ReferenceType                  5865     +5865
builtin_function_or_method     3717     +3717
wrapper_descriptor             3255     +3255
cell                           3022     +3022
getset_descriptor              2964     +2964
type                           2799     +2799
Memory usage after creating instances:
list            5900        +1
set             1378        +1
method          2446        +1
lock              45        +1
Enemy              1        +1
Interactive        1        +1


In [4]:
#Display memory growth before creating instances
print("Memory usage before creating instances:")
objgraph.show_growth()

for i in range(1000):
    no_flyweight_enemy = NoFlyWeightEnemy(name = "small", image=pygame.image.load("./Images/smallEnemy.png"), weapon_name="small", speed=1, default_health=1, pos=(0,0))

# Display memory growth after creating instances
print("Memory usage after creating instances:")
objgraph.show_growth()

Memory usage before creating instances:
dict             10800        +8
list              5904        +4
Constant            10        +4
keyword              6        +4
Attribute            4        +2
ReferenceType     5866        +1
Call                 7        +1
Name                 9        +1
Memory usage after creating instances:
list                 5905        +1
NoFlyWeightEnemy        1        +1


In [5]:
# Display memory growth before creating instances
print("Memory usage before creating instances:")
objgraph.show_growth()

for i in range(1000):
    enemyFlyweight = EnemyFactory.get("small")

# Display memory growth after creating instances
print("Memory usage after creating instances:")
objgraph.show_growth()

Memory usage before creating instances:
ReferenceType     5867        +1
tuple            12287        +1
Memory usage after creating instances:


In [6]:
# Create references graphs
objgraph.show_refs(enemy, filename='memory_graph_enemy.png')
objgraph.show_refs(no_flyweight_enemy, filename='memory_graph_no_flyweight.png')
objgraph.show_refs(enemyFlyweight, filename='memory_graph_flyweight.png')

Graph written to C:\Users\onoratoe\AppData\Local\Temp\objgraph-tvjx3da7.dot (17 nodes)


Image renderer (dot) not found, not doing anything else
Graph written to C:\Users\onoratoe\AppData\Local\Temp\objgraph-b78vfiqg.dot (22 nodes)
Image renderer (dot) not found, not doing anything else
Graph written to C:\Users\onoratoe\AppData\Local\Temp\objgraph-un1znepd.dot (23 nodes)
Image renderer (dot) not found, not doing anything else
